## 引用Keras 套件

In [1]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt
import sys
import numpy as np

Using TensorFlow backend.


## 建立鉴别器(Discriminator)

In [2]:
def build_discriminator():

    model = Sequential()

    img_shape = (28, 28, 1)
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)

## 建立并编译鉴别器

In [3]:
optimizer = Adam(0.0002, 0.5)

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params:

## 建立生成器

In [6]:
latent_dim = 100
def build_generator():

    model = Sequential()

    img_shape = (28, 28, 1)
    model.add(Dense(256, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))
    
    model.summary()

    noise = Input(shape=(latent_dim,))
    img = model(noise)

    return Model(noise, img)

## 堆叠 Generator 與 Discriminator

In [7]:
# 建立生成器
generator = build_generator()

# 使使用噪音生成图片
z = Input(shape=(latent_dim,))
img = generator(z)

# 只训练生成器
discriminator.trainable = False

# 将图片输入给辨别器辨别图片
validity = discriminator(img)

# 堆叠 generator 及 discriminator
# 建立生成器欺骗鉴别器
combined = Model(z, validity)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 256)               1024      
_________________________________________________________________
dense_8 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 512)               2048      
_________________________________________________________________
dense_9 (Dense)              (None, 1024)              525312    
__________

## 建立图像目录

In [ ]:
import os 
os.mkdir('images')

## 图片取样

In [8]:
def sample_images(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)

    # 正规化图片尺度至0  -  1的范围
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/%d.png" % epoch)
    plt.close()

## 训练生成器与鉴别器

In [9]:
batch_size=32
sample_interval=200
epochs = 5000

# 读取MNIST数据
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 将尺度标准化至-1到1
X_train = X_train / 127.5 - 1.
X_train = np.expand_dims(X_train, axis=3)

# 真的图片的答案判定为1生成图片判定为0
valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

for epoch in range(epochs):

    # 建立鉴别器
    # 随机选择部分图片
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[idx]

    noise = np.random.normal(0, 1, (batch_size, latent_dim))

    # 生成新图片
    gen_imgs = generator.predict(noise)

    # 训练鉴别器
    d_loss_real = discriminator.train_on_batch(imgs, valid)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # 训练生成器
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    g_loss = combined.train_on_batch(noise, valid)

    # 印出过程
    print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

    # 产生输出样本
    if epoch % sample_interval == 0:
        sample_images(epoch)



/Users/davidchiu/.pyenv/versions/3.6.2/lib/python3.6/site-packages/keras/engine/training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.748111, acc.: 31.25%] [G loss: 0.722915]
1 [D loss: 0.365462, acc.: 82.81%] [G loss: 0.703483]
2 [D loss: 0.328530, acc.: 87.50%] [G loss: 0.746750]
3 [D loss: 0.330500, acc.: 79.69%] [G loss: 0.857979]
4 [D loss: 0.319957, acc.: 87.50%] [G loss: 0.976308]
5 [D loss: 0.278747, acc.: 95.31%] [G loss: 1.097438]
6 [D loss: 0.239250, acc.: 96.88%] [G loss: 1.261530]
7 [D loss: 0.205997, acc.: 100.00%] [G loss: 1.499375]
8 [D loss: 0.169556, acc.: 98.44%] [G loss: 1.634021]
9 [D loss: 0.125553, acc.: 100.00%] [G loss: 1.729778]
10 [D loss: 0.131501, acc.: 100.00%] [G loss: 1.855536]
11 [D loss: 0.097184, acc.: 100.00%] [G loss: 2.028960]
12 [D loss: 0.086225, acc.: 100.00%] [G loss: 2.097616]
13 [D loss: 0.089019, acc.: 100.00%] [G loss: 2.289205]
14 [D loss: 0.070603, acc.: 100.00%] [G loss: 2.387940]
15 [D loss: 0.054714, acc.: 100.00%] [G loss: 2.443696]
16 [D loss: 0.047027, acc.: 100.00%] [G loss: 2.532602]
17 [D loss: 0.056731, acc.: 100.00%] [G loss: 2.650090]
18 [D loss

155 [D loss: 0.067860, acc.: 98.44%] [G loss: 4.909579]
156 [D loss: 0.136568, acc.: 95.31%] [G loss: 4.724858]
157 [D loss: 0.347890, acc.: 85.94%] [G loss: 4.815576]
158 [D loss: 0.970402, acc.: 62.50%] [G loss: 2.697558]
159 [D loss: 0.297224, acc.: 84.38%] [G loss: 4.263271]
160 [D loss: 0.057804, acc.: 98.44%] [G loss: 5.649585]
161 [D loss: 0.158127, acc.: 95.31%] [G loss: 3.095773]
162 [D loss: 0.209101, acc.: 89.06%] [G loss: 3.557000]
163 [D loss: 0.032354, acc.: 100.00%] [G loss: 4.250163]
164 [D loss: 0.165538, acc.: 93.75%] [G loss: 2.940429]
165 [D loss: 0.113407, acc.: 96.88%] [G loss: 3.220433]
166 [D loss: 0.062709, acc.: 100.00%] [G loss: 4.144823]
167 [D loss: 0.296037, acc.: 84.38%] [G loss: 4.084197]
168 [D loss: 0.053135, acc.: 100.00%] [G loss: 4.673759]
169 [D loss: 0.422998, acc.: 79.69%] [G loss: 2.594287]
170 [D loss: 0.124977, acc.: 93.75%] [G loss: 3.942920]
171 [D loss: 0.108639, acc.: 96.88%] [G loss: 4.072957]
172 [D loss: 0.500051, acc.: 78.12%] [G loss:

307 [D loss: 0.755874, acc.: 40.62%] [G loss: 0.759069]
308 [D loss: 0.695005, acc.: 42.19%] [G loss: 0.770465]
309 [D loss: 0.719945, acc.: 40.62%] [G loss: 0.722162]
310 [D loss: 0.735601, acc.: 45.31%] [G loss: 0.696986]
311 [D loss: 0.663847, acc.: 48.44%] [G loss: 0.786132]
312 [D loss: 0.683176, acc.: 50.00%] [G loss: 0.771415]
313 [D loss: 0.790928, acc.: 34.38%] [G loss: 0.624348]
314 [D loss: 0.691022, acc.: 51.56%] [G loss: 0.688380]
315 [D loss: 0.742966, acc.: 39.06%] [G loss: 0.681413]
316 [D loss: 0.664986, acc.: 54.69%] [G loss: 0.722310]
317 [D loss: 0.724380, acc.: 46.88%] [G loss: 0.700835]
318 [D loss: 0.712351, acc.: 46.88%] [G loss: 0.705585]
319 [D loss: 0.698754, acc.: 50.00%] [G loss: 0.721835]
320 [D loss: 0.749830, acc.: 40.62%] [G loss: 0.649729]
321 [D loss: 0.698887, acc.: 43.75%] [G loss: 0.666935]
322 [D loss: 0.704832, acc.: 45.31%] [G loss: 0.680423]
323 [D loss: 0.745432, acc.: 34.38%] [G loss: 0.633759]
324 [D loss: 0.720062, acc.: 43.75%] [G loss: 0.

454 [D loss: 0.651000, acc.: 64.06%] [G loss: 0.701457]
455 [D loss: 0.676521, acc.: 54.69%] [G loss: 0.697822]
456 [D loss: 0.661443, acc.: 56.25%] [G loss: 0.711531]
457 [D loss: 0.676401, acc.: 53.12%] [G loss: 0.701213]
458 [D loss: 0.646931, acc.: 60.94%] [G loss: 0.692749]
459 [D loss: 0.656325, acc.: 53.12%] [G loss: 0.710754]
460 [D loss: 0.645866, acc.: 62.50%] [G loss: 0.708181]
461 [D loss: 0.660929, acc.: 59.38%] [G loss: 0.726454]
462 [D loss: 0.673565, acc.: 51.56%] [G loss: 0.700310]
463 [D loss: 0.669838, acc.: 56.25%] [G loss: 0.701773]
464 [D loss: 0.652378, acc.: 54.69%] [G loss: 0.674272]
465 [D loss: 0.680228, acc.: 45.31%] [G loss: 0.686981]
466 [D loss: 0.667640, acc.: 50.00%] [G loss: 0.694635]
467 [D loss: 0.658751, acc.: 51.56%] [G loss: 0.667493]
468 [D loss: 0.692985, acc.: 43.75%] [G loss: 0.676338]
469 [D loss: 0.643821, acc.: 56.25%] [G loss: 0.682477]
470 [D loss: 0.656215, acc.: 48.44%] [G loss: 0.686273]
471 [D loss: 0.670755, acc.: 51.56%] [G loss: 0.

...

4281 [D loss: 0.569579, acc.: 64.06%] [G loss: 1.050857]
4282 [D loss: 0.667025, acc.: 57.81%] [G loss: 1.030959]
4283 [D loss: 0.571886, acc.: 78.12%] [G loss: 1.038061]
4284 [D loss: 0.630121, acc.: 62.50%] [G loss: 0.972953]
4285 [D loss: 0.613577, acc.: 68.75%] [G loss: 0.919591]
4286 [D loss: 0.584748, acc.: 73.44%] [G loss: 0.902832]
4287 [D loss: 0.622280, acc.: 59.38%] [G loss: 0.889952]
4288 [D loss: 0.569850, acc.: 70.31%] [G loss: 0.887183]
4289 [D loss: 0.532892, acc.: 71.88%] [G loss: 1.010450]
4290 [D loss: 0.658041, acc.: 57.81%] [G loss: 0.966412]
4291 [D loss: 0.571279, acc.: 68.75%] [G loss: 1.012772]
4292 [D loss: 0.636393, acc.: 70.31%] [G loss: 0.957648]
4293 [D loss: 0.666567, acc.: 56.25%] [G loss: 1.047148]
4294 [D loss: 0.560373, acc.: 73.44%] [G loss: 1.041357]
4295 [D loss: 0.625801, acc.: 67.19%] [G loss: 1.052352]
4296 [D loss: 0.652076, acc.: 59.38%] [G loss: 1.061643]
4297 [D loss: 0.696902, acc.: 54.69%] [G loss: 0.992462]
4298 [D loss: 0.713107, acc.: 5

4431 [D loss: 0.665688, acc.: 59.38%] [G loss: 0.904611]
4432 [D loss: 0.594398, acc.: 64.06%] [G loss: 0.962436]
4433 [D loss: 0.560039, acc.: 76.56%] [G loss: 1.061718]
4434 [D loss: 0.610486, acc.: 71.88%] [G loss: 0.895810]
4435 [D loss: 0.609475, acc.: 67.19%] [G loss: 0.988949]
4436 [D loss: 0.571107, acc.: 71.88%] [G loss: 0.924945]
4437 [D loss: 0.633165, acc.: 65.62%] [G loss: 0.926554]
4438 [D loss: 0.525761, acc.: 71.88%] [G loss: 0.995836]
4439 [D loss: 0.643872, acc.: 62.50%] [G loss: 0.976844]
4440 [D loss: 0.636953, acc.: 65.62%] [G loss: 0.970777]
4441 [D loss: 0.725653, acc.: 51.56%] [G loss: 1.013736]
4442 [D loss: 0.659785, acc.: 70.31%] [G loss: 1.007975]
4443 [D loss: 0.636356, acc.: 62.50%] [G loss: 1.027375]
4444 [D loss: 0.610468, acc.: 68.75%] [G loss: 0.997017]
4445 [D loss: 0.593403, acc.: 70.31%] [G loss: 0.960034]
4446 [D loss: 0.558593, acc.: 75.00%] [G loss: 1.008194]
4447 [D loss: 0.657117, acc.: 59.38%] [G loss: 1.002467]
4448 [D loss: 0.673748, acc.: 5

4578 [D loss: 0.623126, acc.: 65.62%] [G loss: 1.055363]
4579 [D loss: 0.601568, acc.: 67.19%] [G loss: 0.954503]
4580 [D loss: 0.670660, acc.: 56.25%] [G loss: 0.948277]
4581 [D loss: 0.631728, acc.: 64.06%] [G loss: 0.998671]
4582 [D loss: 0.599354, acc.: 73.44%] [G loss: 1.038298]
4583 [D loss: 0.572240, acc.: 82.81%] [G loss: 0.928411]
4584 [D loss: 0.537517, acc.: 76.56%] [G loss: 1.009141]
4585 [D loss: 0.639824, acc.: 60.94%] [G loss: 1.016483]
4586 [D loss: 0.618701, acc.: 73.44%] [G loss: 0.938743]
4587 [D loss: 0.599729, acc.: 67.19%] [G loss: 0.858785]
4588 [D loss: 0.615975, acc.: 64.06%] [G loss: 0.983194]
4589 [D loss: 0.651963, acc.: 62.50%] [G loss: 0.885031]
4590 [D loss: 0.544450, acc.: 71.88%] [G loss: 0.865662]
4591 [D loss: 0.623042, acc.: 68.75%] [G loss: 0.925354]
4592 [D loss: 0.642569, acc.: 57.81%] [G loss: 0.950395]
4593 [D loss: 0.602972, acc.: 70.31%] [G loss: 0.925790]
4594 [D loss: 0.582437, acc.: 68.75%] [G loss: 0.957327]
4595 [D loss: 0.628374, acc.: 6

4725 [D loss: 0.579328, acc.: 70.31%] [G loss: 0.942702]
4726 [D loss: 0.590766, acc.: 70.31%] [G loss: 0.877075]
4727 [D loss: 0.665356, acc.: 59.38%] [G loss: 0.980443]
4728 [D loss: 0.654499, acc.: 68.75%] [G loss: 0.987288]
4729 [D loss: 0.616034, acc.: 59.38%] [G loss: 0.871004]
4730 [D loss: 0.586114, acc.: 71.88%] [G loss: 0.888765]
4731 [D loss: 0.624904, acc.: 59.38%] [G loss: 0.844867]
4732 [D loss: 0.562450, acc.: 71.88%] [G loss: 0.891920]
4733 [D loss: 0.598748, acc.: 67.19%] [G loss: 0.989588]
4734 [D loss: 0.626483, acc.: 70.31%] [G loss: 0.958309]
4735 [D loss: 0.669290, acc.: 59.38%] [G loss: 0.897363]
4736 [D loss: 0.566425, acc.: 67.19%] [G loss: 1.002838]
4737 [D loss: 0.496672, acc.: 82.81%] [G loss: 0.928117]
4738 [D loss: 0.576261, acc.: 71.88%] [G loss: 0.960639]
4739 [D loss: 0.589041, acc.: 70.31%] [G loss: 0.963086]
4740 [D loss: 0.685631, acc.: 60.94%] [G loss: 0.923227]
4741 [D loss: 0.649420, acc.: 67.19%] [G loss: 1.106225]
4742 [D loss: 0.606357, acc.: 6

4869 [D loss: 0.614727, acc.: 65.62%] [G loss: 0.999185]
4870 [D loss: 0.585149, acc.: 67.19%] [G loss: 0.949513]
4871 [D loss: 0.659019, acc.: 59.38%] [G loss: 1.019336]
4872 [D loss: 0.628845, acc.: 64.06%] [G loss: 0.943940]
4873 [D loss: 0.657269, acc.: 64.06%] [G loss: 0.890751]
4874 [D loss: 0.580717, acc.: 73.44%] [G loss: 0.893676]
4875 [D loss: 0.657149, acc.: 54.69%] [G loss: 0.970283]
4876 [D loss: 0.585823, acc.: 70.31%] [G loss: 1.018455]
4877 [D loss: 0.570084, acc.: 76.56%] [G loss: 1.079851]
4878 [D loss: 0.657592, acc.: 50.00%] [G loss: 0.974251]
4879 [D loss: 0.619412, acc.: 70.31%] [G loss: 0.986013]
4880 [D loss: 0.576916, acc.: 75.00%] [G loss: 0.938426]
4881 [D loss: 0.585175, acc.: 65.62%] [G loss: 1.054089]
4882 [D loss: 0.635870, acc.: 62.50%] [G loss: 1.030909]
4883 [D loss: 0.603892, acc.: 57.81%] [G loss: 0.987685]
4884 [D loss: 0.628213, acc.: 67.19%] [G loss: 1.067871]
4885 [D loss: 0.611863, acc.: 67.19%] [G loss: 0.967864]
4886 [D loss: 0.634907, acc.: 5